<a href="https://colab.research.google.com/github/willburr/humorous-headlines/blob/bert-embeddings/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Coursework coding instructions (please also see full coursework spec)

Please choose if you want to do either Task 1 or Task 2. You should write your report about one task only.

For the task you choose you will need to do two approaches:
  - Approach 1, which can use use pre-trained embeddings / models
  - Approach 2, which should not use any pre-trained embeddings or models
We should be able to run both approaches from the same colab file

#### Running your code:
  - Your models should run automatically when running your colab file without further intervention
  - For each task you should automatically output the performance of both models
  - Your code should automatically download any libraries required

#### Structure of your code:
  - You are expected to use the 'train', 'eval' and 'model_performance' functions, although you may edit these as required
  - Otherwise there are no restrictions on what you can do in your code

#### Documentation:
  - You are expected to produce a .README file summarising how you have approached both tasks

#### Reproducibility:
  - Your .README file should explain how to replicate the different experiments mentioned in your report

Good luck! We are really looking forward to seeing your reports and your model code!

# Project  Set-up

The code blocks below provide the imports and necessaryset-up steps to run later sections. We import necessary libraries, set pytorch to use the GPU and load and unzip the data.

In [1]:
# Imports

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import Dataset, random_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import torch.optim as optim
import codecs
import tqdm

In [2]:
# Setting random seed and device
SEED = 1

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

## Downloading and unzipping the data

TODO: Eventually remove drive import and replace with !wget (Might be faster to just do this anyway) 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Unzip data
!unzip "/content/drive/MyDrive/cw/task-2-20210202T162438Z-001.zip"

Archive:  /content/drive/MyDrive/cw/task-2-20210202T162438Z-001.zip
  inflating: task-2/.DS_Store        
  inflating: task-2/dev.csv          
  inflating: task-2/train.csv        


In [5]:
# Load data
train_df = pd.read_csv('task-2/train.csv')
test_df = pd.read_csv('task-2/dev.csv')

In [6]:
# Take a look at the data
train_df.head()

,id,original1,edit1,grades1,meanGrade1,original2,edit2,grades2,meanGrade2,label
0,10920-9866,""" Gene Cernan , Last <Astronaut/> on the Moon ...",Dancer,1113,1.2,""" Gene Cernan , Last Astronaut on the Moon , <...",impregnated,30001,0.8,1
1,3176-10722,""" I 'm done "" : Fed up with California , some ...",vagrants,1200,0.6,""" I 'm done "" : Fed up with <California/> , so...",pancakes,10110,0.6,0
2,3176-3702,""" I 'm done "" : Fed up with California , some ...",vagrants,1200,0.6,""" I 'm done "" : Fed up with <California/> , so...",life,2,0.4,1
3,10722-3702,""" I 'm done "" : Fed up with <California/> , so...",pancakes,10110,0.6,""" I 'm done "" : Fed up with <California/> , so...",life,2,0.4,1
4,12282-2083,""" Our expectations of what civic engagement lo...",imagine,0,0.0,""" Our expectations of what civic engagement <l...",smells,100220010,0.6,2


## Download and unzip GLOVE Embeddings

TODO: Do we bother using GLOVE at all? May be interesting in report to compare GLOVE vs BERT embeddings?

In [7]:
# You will need to download any word embeddings required for your code, e.g.:

# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip

# For any packages that Colab does not provide auotmatically you will also need to install these below, e.g.:

#! pip install torch

In [8]:
!unzip '/content/drive/MyDrive/cw/glove.6B.zip'

Archive:  /content/drive/MyDrive/cw/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


## Download and install Transformers library

In [9]:
!pip install transformers

     |████████████████████████████████| 1.8MB 5.8MB/s 
     |████████████████████████████████| 890kB 39.3MB/s 
     |████████████████████████████████| 3.2MB 38.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=59637580cb82a5f0373d00b200c2903265e36e1b11bda6f5aa19fac31d3fd357
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## Training Evaluation Utility Functions

In [10]:
# We define our training loop
def train(train_iter, dev_iter, model, number_epoch, optimizer, scheduler=None):
    """
    Training loop for the model, which calls on eval to evaluate after each epoch
    """

    print("Training model.")

    for epoch in range(1, number_epoch+1):
        
        model.train()
        
        epoch_loss = 0
        epoch_correct = 0
        no_observations = 0  # Observations used for training so far

        for batch in train_iter:
            feature, target = batch
            feature, target = feature.to(device), target.to(device)

            # for RNN:
            model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # model.hidden = model.init_hidden()

            predictions = model(feature).squeeze(1)
            optimizer.zero_grad()
            loss = loss_fn(predictions, target)

            correct, __ = model_performance(np.argmax(predictions.detach().cpu().numpy(), axis=1), target.detach().cpu().numpy())

            loss.backward()
            optimizer.step()

            if scheduler:
              scheduler.step()

            epoch_loss += loss.item()*target.shape[0]
            epoch_correct += correct

        valid_loss, valid_acc, __, __ = eval(dev_iter, model)

        epoch_loss, epoch_acc = epoch_loss / no_observations, epoch_correct / no_observations
        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train Accuracy: {epoch_acc:.2f} | \
        Val. Loss: {valid_loss:.2f} | Val. Accuracy: {valid_acc:.2f} |')

In [11]:
# We evaluate performance on our dev set
def eval(data_iter, model):
    """
    Evaluating model performance on the dev set
    """
    model.eval()
    epoch_loss = 0
    epoch_correct = 0
    pred_all = []
    trg_all = []
    no_observations = 0

    with torch.no_grad():
        for batch in data_iter:
            feature, target = batch

            feature, target = feature.to(device), target.to(device)

            # for RNN:
            model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # model.hidden = model.init_hidden()

            predictions = model(feature).squeeze(1)
            loss = loss_fn(predictions, target)

            # We get the mse
            pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
            correct, __ = model_performance(np.argmax(pred, axis=1), trg)

            epoch_loss += loss.item()*target.shape[0]
            epoch_correct += correct
            pred_all.extend(pred)
            trg_all.extend(trg)

    return epoch_loss/no_observations, epoch_correct/no_observations, np.array(pred_all), np.array(trg_all)

In [12]:
# How we print the model performance
def model_performance(output, target, print_output=False):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    correct_answers = (output == target)
    correct = sum(correct_answers)
    acc = np.true_divide(correct,len(output))

    if print_output:
        print(f'| Acc: {acc:.2f} ')

    return correct, acc

# Data augmentation and Vocab Preparation

## Data augmentation

In [13]:
import re

def substitute(sentence, edit):
  open_pos = sentence.find('<')
  close_pos = sentence.find('>')
  sub = sentence.replace(sentence[open_pos: close_pos + 1], edit)
  return sub

def get_edited_df(df):
  edited1 = df.apply(lambda x:substitute(x['original1'], x['edit1']), axis=1)
  edited2 = df.apply(lambda x:substitute(x['original2'], x['edit2']), axis=1)
  combined = list(zip(edited1, edited2))
  return pd.DataFrame(combined, columns=['edited1', 'edited2'])

edited_train_df = get_edited_df(train_df)
edited_test_df = get_edited_df(test_df)

edited_train_df.head()

,edited1,edited2
0,""" Gene Cernan , Last Dancer on the Moon , Dies...",""" Gene Cernan , Last Astronaut on the Moon , i..."
1,""" I 'm done "" : Fed up with California , some ...",""" I 'm done "" : Fed up with pancakes , some co..."
2,""" I 'm done "" : Fed up with California , some ...",""" I 'm done "" : Fed up with life , some conser..."
3,""" I 'm done "" : Fed up with pancakes , some co...",""" I 'm done "" : Fed up with life , some conser..."
4,""" Our expectations of what civic engagement lo...",""" Our expectations of what civic engagement sm..."


## Define Vocab Class

TODO: probs move this up to top and have functions for using GLOVE, BERT and our own

In [14]:
class Vocabulary(object):
  """Data structure representing the vocabulary of a corpus."""
  def __init__(self):
    # Mapping from tokens to integers
    self._word2idx = {}

    # Reverse-mapping from integers to tokens
    self.idx2word = []

  def word2idx(self, word, default=None):
    """Returns the integer ID of the word or default if not found."""
    return self._word2idx.get(word, default)

  def add_word(self, word):
    """Adds the `word` into the vocabulary."""
    if word not in self._word2idx:
      self.idx2word.append(word)
      self._word2idx[word] = len(self.idx2word) - 1

  def build_from_list(self, words):
    for word in words:
      self.add_word(word)

  def build_from_file(self, fname):
    """Builds a vocabulary from a given corpus file."""
    with open(fname) as f:
      for line in f:
        words = line.strip().split()
        for word in words:
          self.add_word(word)

  def convert_idxs_to_words(self, idxs):
    """Converts a list of indices to words."""
    return ' '.join(self.idx2word[idx] for idx in idxs)

  def convert_words_to_idxs(self, words):
    """Converts a list of words to a list of indices."""
    return [self.word2idx(w) for w in words]

  def __len__(self):
    """Returns the size of the vocabulary."""
    return len(self.idx2word)
  
  def __repr__(self):
    return "Vocabulary with {} items".format(self.__len__())

## Vocab Preparation

In [15]:
# To create our vocab (TODO: and return Vocabulary object)
def create_vocab(data):
    """
    Creating a corpus of all the tokens used
    """
    tokenized_corpus = [] # Let us put the tokenized corpus in a list

    for sentence_pair in data:
        tokenized_sentence_pair = []
        for sentence in sentence_pair:
            tokenized_sentence = []

            for token in sentence.split(' '): # simplest split is

                tokenized_sentence.append(token.lower())

            tokenized_sentence_pair.append(tokenized_sentence)
        tokenized_corpus.append(tokenized_sentence_pair)

    # Create single list of all vocabulary
    vocabulary = []  # Let us put all the tokens (mostly words) appearing in the vocabulary in a list

    for sentence_pair in tokenized_corpus:

        for sentence in sentence_pair:

            for token in sentence:

                if token.lower() not in vocabulary:

                    vocabulary.append(token.lower())

    return vocabulary, tokenized_corpus

TODO: Potentially break up code blocks a bit more?

The below code block loads in the GLOVE embeddings.
TODO: We can move this into the Vocabulary class

## Load the Dataset

### Utility functions

In [16]:
# We create a Dataset so we can create minibatches
class Task2Dataset(Dataset):

    def __init__(self, train_data, labels):
        self.x_train = train_data
        self.y_train = labels

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, item):
        return self.x_train[item], self.y_train[item]

### Creating Dataloaders

In [39]:
def flip_label(label):
  if label == 1:
    return 2
  elif label == 2:
    return 1
  else: 
    return label

def create_dataloaders(feature, labels, collate_fn, train_proportion=0.8, batch_size=32, flip=True):
    # 'feature' is a list of lists, each containing embedding IDs for word tokens
    train_and_dev = Task2Dataset(feature, labels)

    train_examples = round(len(train_and_dev)*train_proportion)
    dev_examples = len(train_and_dev) - train_examples

    train_dataset, dev_dataset = random_split(train_and_dev,
                                              (train_examples,
                                                dev_examples))

    if flip:
        # Add the reverse examples to the training set to create more training dara
        flipped_train_dataset = []
        for r in train_dataset:
          flipped_train_dataset.append((list(reversed(r[0])), flip_label(r[1])))
        train_dataset += flipped_train_dataset

    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size, collate_fn=collate_fn)
    dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size, collate_fn=collate_fn)
    
    return train_loader, dev_loader

# Approach 1: Using pre-trained representations

## 1.1 Using pre-trained (GloVe) embeddings with a BiLSTM model

In [18]:
## Approach 1 code, using functions defined above:

# We set our training data and test data
training_data = edited_train_df.values.tolist()
test_data = edited_test_df.values.tolist()

# Creating word vectors
training_vocab, training_tokenized_corpus = create_vocab(training_data)
test_vocab, test_tokenized_corpus = create_vocab(test_data)

# Creating joint vocab from test and train
# TODO: We can optimise to just use the previous two:
joint_vocab, joint_tokenized_corpus = create_vocab(training_data + test_data)

print("Vocab created.")

Vocab created.


In [19]:
# We create representations for our tokens
wvecs = [] # word vectors

vocab = Vocabulary()
# This is a large file, it will take a while to load in the memory!
with codecs.open('glove.6B.300d.txt', 'r','utf-8') as f:
  index = 1
  for line in f.readlines():
    # Ignore the first line - first line typically contains vocab, dimensionality
    if len(line.strip().split()) > 3:
      word = line.strip().split()[0]
      if word in joint_vocab:
          (word, vec) = (word,
                     list(map(float,line.strip().split()[1:])))
          wvecs.append(vec)
          vocab.add_word(word)


wvecs = np.array(wvecs)

vectorized_seqs = [[[vocab.word2idx(tok) for tok in sen if tok in vocab._word2idx] for sen in seq] for seq in training_tokenized_corpus]


In [20]:
# Used for collating our observations into minibatches:
def collate_fn_padd(batch):
    '''
    We add padding to our minibatches and create tensors for our model
    '''
    
    batch_labels = [l for f, l in batch]
    batch_features = [f for f, l in batch]

    batch_features_len = [[len(s) for s in f] for f, l in batch]

    seq_tensor = torch.zeros((2, len(batch), np.max(batch_features_len))).long()

    # Shape of seq_tensor is batch_size x max_feature_len
    # It should be batch_size x 2 x max_feature_len

    for idx, (seq, seqlens) in enumerate(zip(batch_features, batch_features_len)):
        for i in range(2):
            seq_tensor[i, idx, :seqlens[i]] = torch.LongTensor(seq[i])

    batch_labels = torch.LongTensor(batch_labels)

    return seq_tensor, batch_labels

In [21]:
feature = vectorized_seqs
labels = list(train_df['label'])

train_loader, dev_loader = create_dataloaders(feature, labels, collate_fn_padd)

print("Dataloaders created.")

Dataloaders created.


In [22]:
class BiLSTM_classification(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(BiLSTM_classification, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2label = nn.Linear(hidden_dim * 4, 3)

    def forward(self, sentence):
        embedding1 = self.embedding(sentence[0])
        embedding1 = embedding1.permute(1, 0, 2)

        embedding2 = self.embedding(sentence[1])
        embedding2 = embedding2.permute(1, 0, 2)

        lstm_out_1, _ = self.lstm(embedding1.view(len(embedding1), -1, self.embedding_dim))
        lstm_out_2, _ = self.lstm(embedding2.view(len(embedding2), -1, self.embedding_dim))
        # concat both lstm_out_1 and lstm_out_2 and give to our linear layer, think we need
        # output shape of lstm_ out is 2 * hidden_size as we do it in both directions
 
        # lstm_out_1[-1] are both batch_size x (2 * hidden_size)
        # do we concat so it's (2*batch_size) x (2 * hidden_size) or batch_size x (2*(2 * hidden_size))? - I chose the latter
        out = self.hidden2label(torch.cat((lstm_out_1[-1], lstm_out_2[-1]), 1))
        return out

TODO: Separate out dataset stuff and model stuff below and put data loading stuff in section above. 

In [23]:
# Number of epochs
epochs = 16

INPUT_DIM = len(vocab) + 1
HIDDEN_DIM = 400 # Higher appears to yield better val acc
EMBEDDING_DIM = 300 # Higher yields better val acc

## TODO: Figure out network dimensions
model = BiLSTM_classification(EMBEDDING_DIM, HIDDEN_DIM, INPUT_DIM)
print("Model initialised.")

model.to(device)
# We provide the model with our embeddings
model.embedding.weight.data[1:].copy_(torch.from_numpy(wvecs))


Model initialised.


tensor([[ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        [-0.2554, -0.2572,  0.1317,  ..., -0.2329, -0.1223,  0.3550],
        [-0.1256,  0.0136,  0.1031,  ..., -0.3422, -0.0224,  0.1368],
        ...,
        [ 0.5028, -0.1677,  0.0193,  ...,  0.4927,  0.6148,  0.0180],
        [ 0.1070, -0.0379,  0.2672,  ...,  0.2112,  0.3525,  0.1777],
        [-0.1868, -0.2897, -0.0846,  ...,  0.2076,  0.5071,  0.3187]],
       device='cuda:0')

In [24]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

optimizer = torch.optim.Adam(model.parameters())

train(train_loader, dev_loader, model, epochs, optimizer)

Training model.
| Epoch: 01 | Train Loss: 0.96 | Train Accuracy: 0.45 |         Val. Loss: 0.96 | Val. Accuracy: 0.45 |
| Epoch: 02 | Train Loss: 0.85 | Train Accuracy: 0.60 |         Val. Loss: 1.02 | Val. Accuracy: 0.57 |
| Epoch: 03 | Train Loss: 0.53 | Train Accuracy: 0.78 |         Val. Loss: 1.12 | Val. Accuracy: 0.59 |
| Epoch: 04 | Train Loss: 0.31 | Train Accuracy: 0.88 |         Val. Loss: 1.40 | Val. Accuracy: 0.58 |
| Epoch: 05 | Train Loss: 0.17 | Train Accuracy: 0.94 |         Val. Loss: 2.13 | Val. Accuracy: 0.59 |
| Epoch: 06 | Train Loss: 0.11 | Train Accuracy: 0.96 |         Val. Loss: 2.55 | Val. Accuracy: 0.58 |
| Epoch: 07 | Train Loss: 0.06 | Train Accuracy: 0.98 |         Val. Loss: 3.33 | Val. Accuracy: 0.59 |
| Epoch: 08 | Train Loss: 0.03 | Train Accuracy: 0.99 |         Val. Loss: 4.20 | Val. Accuracy: 0.61 |
| Epoch: 09 | Train Loss: 0.03 | Train Accuracy: 0.99 |         Val. Loss: 3.97 | Val. Accuracy: 0.60 |
| Epoch: 10 | Train Loss: 0.04 | Train Accuracy:

## 1.2 Using BERT embeddings
 

In [25]:
from transformers import DistilBertTokenizer, DistilBertModel 

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [26]:
training_data = edited_train_df.values.tolist()
test_data = edited_test_df.values.tolist()

training_labels = train_df['label'].values.tolist()

input_id_pairs = []

for (sentence1, sentence2) in training_data:
    encoded_sentence1 = tokenizer.encode(sentence1)
    encoded_sentence2 = tokenizer.encode(sentence2)
    
    # Add the encoded sentence to the list.
    input_id_pairs.append([encoded_sentence1, encoded_sentence2])

In [27]:
# Used for collating our observations into minibatches:

# Note that this one adds an attention mask (used to tell BERT to ignore padding)
def collate_fn_padd_BERT_embeddings(batch):
    '''
    We add padding to our minibatches and create tensors for our model
    '''
    batch_labels = [l for f, l in batch]
    batch_features = [f for f, l in batch]

    batch_features_len = [[len(s) for s in f] for f, l in batch]

    seq_tensor = torch.zeros((2, len(batch), np.max(batch_features_len))).long()
    mask_tensor = torch.zeros((2, len(batch), np.max(batch_features_len))).long()

    # Shape of seq_tensor is batch_size x max_feature_len
    # It should be batch_size x 2 x max_feature_len

    for idx, (seq, seqlens) in enumerate(zip(batch_features, batch_features_len)):
        for i in range(2):
            seq_tensor[i, idx, :seqlens[i]] = torch.LongTensor(seq[i])
            mask_tensor[i, idx, :seqlens[i]] = torch.ones(seqlens[i])

    batch_labels = torch.LongTensor(batch_labels)

    feature_tensor = torch.stack([seq_tensor, mask_tensor], dim=1)
    return feature_tensor, batch_labels

In [28]:
train_loader, dev_loader = create_dataloaders(input_id_pairs, training_labels, collate_fn_padd_BERT_embeddings)

print("Dataloaders created.")

Dataloaders created.


In [29]:
class BiLSTM_BERT_classification(nn.Module):

    def __init__(self, hidden_dim):
        super(BiLSTM_BERT_classification, self).__init__()
        self.hidden_dim = hidden_dim
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased',
            output_hidden_states = True, 
        )
        # The word embedding dimension given by concatenating last 4 hidden 
        # layers of BERT
        self.embedding_dim = 3072

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(self.embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2label = nn.Linear(hidden_dim * 4, 3)

    def compute_embeddings(self, feature):
        b_input_ids = feature[0].to(device)
        b_input_mask = feature[1].to(device)

        with torch.no_grad():
            # We aren't trying to train BERT here, only use its embeddings
            self.bert.eval()
            outputs = self.bert(
                b_input_ids,
                attention_mask=b_input_mask
            )
            hidden_states = outputs.hidden_states
        
        return torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1)

    def forward(self, sentences):
        embedding1 = self.compute_embeddings(sentences[0])
        embedding1 = embedding1.permute(1, 0, 2)
        embedding2 = self.compute_embeddings(sentences[1])
        embedding2 = embedding2.permute(1, 0, 2)

        lstm_out_1, _ = self.lstm(embedding1.view(len(embedding1), -1, self.embedding_dim))
        lstm_out_2, _ = self.lstm(embedding2.view(len(embedding2), -1, self.embedding_dim))
        # concat both lstm_out_1 and lstm_out_2 and give to our linear layer, think we need
        # output shape of lstm_ out is 2 * hidden_size as we do it in both directions
 
        # lstm_out_1[-1] are both batch_size x (2 * hidden_size)
        # do we concat so it's (2*batch_size) x (2 * hidden_size) or batch_size x (2*(2 * hidden_size))? - I chose the latter
        out = self.hidden2label(torch.cat((lstm_out_1[-1], lstm_out_2[-1]), 1))
        return out

In [30]:
# Number of epochs
epochs = 16

HIDDEN_DIM = 400 # Higher appears to yield better val acc

## TODO: Figure out network dimensions
model = BiLSTM_BERT_classification(HIDDEN_DIM)
print("Model initialised.")

model = model.to(device)


Model initialised.


In [31]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

optimizer = torch.optim.Adam(model.parameters())

train(train_loader, dev_loader, model, epochs, optimizer)

Training model.
| Epoch: 01 | Train Loss: 0.96 | Train Accuracy: 0.49 |         Val. Loss: 0.92 | Val. Accuracy: 0.53 |


KeyboardInterrupt: ignored

## 1.3 Fine-tuning BERT

The following code is basically just an initial attempt to fine-tune BERT for the task. 
A good amount of the code here is either inspired by or, in the case of the training loop, literally taken from this link: 
https://github.com/aniruddhachoudhury/BERT-Tutorials/blob/master/Blog%202/BERT_Fine_Tuning_Sentence_Classification.ipynb

I know it needs a refactor. I just want it to work. 

I think we could also try to take some inspiration from the following two resources: 

- https://medium.com/@theoliao1998/humor-detection-with-a-bert-regressor-9dc0a821c294
- https://arxiv.org/pdf/2004.12765.pdf
- https://github.com/ceshine/pytorch-pretrained-BERT/blob/master/notebooks/Next%20Sentence%20Prediction.ipynb

In [40]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

class HumourBert(nn.Module):
    def __init__(self):
        super(HumourBert, self).__init__()
        self.bert = DistilBertForSequenceClassification.from_pretrained(
            'distilbert-base-uncased', 
            num_labels = 3,
            output_attentions = False,
            output_hidden_states = False
        )
    
    def forward(self, feature):
        b_input_ids = feature[0].to(device)
        b_input_mask = feature[1].to(device)
        outputs = self.bert(
            b_input_ids,
            attention_mask=b_input_mask
        )
        return outputs.logits


model = HumourBert().to(device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [41]:
# We set our training data and test data
training_data = edited_train_df.values.tolist()
test_data = edited_test_df.values.tolist()

training_labels = train_df['label'].values.tolist()

In [42]:
input_id_pairs = []

cls = tokenizer.encode('[CLS]')
sep = tokenizer.encode('[SEP]')

for (sentence1, sentence2) in training_data:
    encoded_sentence1 = tokenizer.encode(sentence1, add_special_tokens=False)
    encoded_sentence2 = tokenizer.encode(sentence2, add_special_tokens=False)
    
    # Add the encoded sentence to the list.
    input_id_pairs.append([encoded_sentence1, encoded_sentence2])

In [43]:
def collate_fn_BERT(batch):
    '''
    We add padding to our minibatches and create tensors for our model
    '''
    batch_labels = [l for f, l in batch]
    # Add the CLS and SEP tokens here
    batch_features = [cls + s1 + sep + s2 for (s1, s2), l in batch]

    batch_features_len = [len(f) for f in batch_features]

    seq_tensor = torch.zeros((len(batch), max(batch_features_len))).long()
    mask_tensor = torch.zeros((len(batch), max(batch_features_len))).long()

    for idx, (seq, seqlen) in enumerate(zip(batch_features, batch_features_len)):
        seq_tensor[idx, :seqlen] = torch.LongTensor(seq)
        mask_tensor[idx, :seqlen] = torch.ones(seqlen)

    batch_labels = torch.LongTensor(batch_labels)

    # Put sequence and tensor together
    feature_tensor = torch.stack([seq_tensor, mask_tensor])
    
    return feature_tensor, batch_labels

In [44]:
train_loader, dev_loader = create_dataloaders(input_id_pairs, training_labels, collate_fn_BERT, flip=False)

print("Dataloaders created.")

Dataloaders created.


In [45]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 1e-5,
                  eps = 1e-8
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 16

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

train(train_loader, dev_loader, model, epochs, optimizer, scheduler)

# Approach 2: No pre-trained representations

TODO: Make code work with sentence pairs
TODO: Can probably make below function part of Vocabulary()

In [ ]:
# def corpus_to_tensor(_vocab, corpus):
#   # Final token indices
#   idxs = []
  
#   for sent in corpus:
#         sent = f"<s> {line} </s>"
#         # Split from whitespace and add sentence markers
#         idxs.extend(_vocab.convert_words_to_idxs(line.split()))
#   return torch.LongTensor(idxs)

## Create word-index mappings

In [ ]:
vocab = Vocabulary()

# 0-padding token
vocab.add_word('<pad>')
# sentence start
vocab.add_word('<s>')
# sentence end
vocab.add_word('</s>')
# Unknown words
vocab.add_word('<unk>')
vocab.build_from_list(joint_vocab)
print(vocab)
print(vocab.convert_words_to_idxs('the dancer is on the moon'.split()))

## Prepare data
TODO: Reduce duplication

In [ ]:
vectorized_seqs = [[[vocab._word2idx[tok] for tok in sen if tok in vocab._word2idx] for sen in seq] for seq in training_tokenized_corpus]

In [ ]:
train_loader, dev_loader = create_dataloaders(input_id_pairs, training_labels, collate_fn_padd)

print("Dataloaders created.")

## Bi-LSTM model

In [ ]:
# Number of epochs
epochs = 16

INPUT_DIM = len(vocab) + 1
HIDDEN_DIM = 400 # Higher appears to yield better val acc
EMBEDDING_DIM = 300 # Higher yields better val acc
BATCH_SIZE = 32

## TODO: Figure out network dimensions
model = BiLSTM_classification(EMBEDDING_DIM, HIDDEN_DIM, INPUT_DIM, BATCH_SIZE, device)
print("Model initialised.")
model.to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

optimizer = torch.optim.Adam(model.parameters())

train(train_loader, dev_loader, model, epochs)

## LSTM model with attention

Below code was inspired by the following:


*   https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
*   https://medium.com/intel-student-ambassadors/implementing-attention-models-in-pytorch-f947034b3e66
* https://www.kaggle.com/yshubham/simple-lstm-for-text-classification-with-attention
* https://iopscience.iop.org/article/10.1088/1742-6596/1207/1/012008






In [ ]:
class AttnLSTM(nn.Module):
  
  def __init__(self, embedding_dim, hidden_dim, output_dim, vocab_size, dropout_prop=0.1):
    super(AttnLSTM, self).__init__()
    
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dimm
    self.output_dim = output_dim
    self.dropout_prop = dropout_prop
    
    self.embedding = nn.Embedding(
      num_embeddings=self.vocab_size, embedding_dim=self.embedding_dim,
      padding_idx=0)
    self.attn = nn.Linear(embedding_dim + hidden_dim, 1)
    self.dropout = nn.Dropout(self.dropout_prop)
    self.hidden = self.init_hidden()

    self.lstm = nn.LSTM(hidden_dim + vocab_size, output_size)
    self.final = nn.Linear(hideen_dim, output_dim)
  
  def init_hidden(self):
    return (torch.zeros(1, 1, self.output_size),
      torch.zeros(1, 1, self.output_size))
  
  def forward(self, input, prev_out):
    embedded = self.embedding(input).view(1, 1, -1)
    embedded = self.dropout(embedded)

    # Use MLP attention
    attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], self.hidden[0]), 1)), dim=1)
    
    # TODO: How tf does this work?
    attn_applied = torch.bmm(attn_weights.unsqueeze(0), 
                             prev_out.view(1, -1, self.hidden_dim))
    
    lstm_in = torch.cat((attn_applied, embedded), dim = 1)
    
    output, self.hidden = self.lstm(lstm_in.unsqueeze(0), self.hidden)
    
    output = self.final(output)

    ## TODO: What about softmax / sigmoid?
    # output = F.log_softmax(self.out(output[0]), dim=1)
    
    return output

## Training BERT from scratch

following code adapted from:
https://huggingface.co/blog/how-to-train

Tokenizer docs: https://github.com/huggingface/tokenizers/tree/master/bindings/pythonhttps://github.com/huggingface/tokenizers/tree/master/bindings/python


In [ ]:
# TODO: Move to top
from tokenizers import ByteLevelBPETokenizer

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# TODO: tokenize text with filepath? pass in train.csv? convert to txt?

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# produces a vocab.json and a merges.txt

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification()

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

## Pre-provided code for approach 2:

In [ ]:
train_and_dev = train_df['edit1']

training_data, dev_data, training_y, dev_y = train_test_split(train_df['edit1'], train_df['label'],
                                                                        test_size=(1-train_proportion),
                                                                        random_state=42)

# We train a Tf-idf model
count_vect = CountVectorizer(stop_words='english')
train_counts = count_vect.fit_transform(training_data)
transformer = TfidfTransformer().fit(train_counts)
train_counts = transformer.transform(train_counts)
naive_model = MultinomialNB().fit(train_counts, training_y)

# Train predictions
predicted_train = naive_model.predict(train_counts)

# Calculate Tf-idf using train and dev, and validate on dev:
test_and_test_counts = count_vect.transform(train_and_dev)
transformer = TfidfTransformer().fit(test_and_test_counts)

test_counts = count_vect.transform(dev_data)

test_counts = transformer.transform(test_counts)

# Dev predictions
predicted = naive_model.predict(test_counts)

# We run the evaluation:
print("\nTrain performance:")

sse, mse = model_performance(predicted_train, training_y, True)

print("\nDev performance:")
sse, mse = model_performance(predicted, dev_y, True)

#### Baseline for task 2

In [ ]:
# Baseline for the task
pred_baseline = torch.zeros(len(dev_y)) + 1  # 1 is most common class
print("\nBaseline performance:")
sse, mse = model_performance(pred_baseline, torch.tensor(dev_y.values), True)